In [1]:
import time
import re
import pandas as pd

from rapidfuzz import process, fuzz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
class CollegeScraper:
    def __init__(self, url, wait=6, scroll_pause=1.2):
        self.url = url
        self.wait = wait
        self.scroll_pause = scroll_pause
        self.data = []

    def _init_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument("--disable-blink-features=AutomationControlled")
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    def scrape(self):
        driver = self._init_driver()
        driver.get(self.url)
        time.sleep(self.wait)

        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.listing-table")))

        scroll_container = driver.find_element(By.CSS_SELECTOR, "div.table-wrapper")

        last_count, stable_rounds = 0, 0

        while True:
            rows = driver.find_elements(By.CSS_SELECTOR, "table.listing-table tbody tr")
            current_count = len(rows)

            if current_count == last_count:
                stable_rounds += 1
            else:
                stable_rounds = 0

            last_count = current_count

            if stable_rounds >= 6:
                break

            driver.execute_script(
                "arguments[0].scrollTop = arguments[0].scrollHeight",
                scroll_container
            )
            time.sleep(self.scroll_pause)

        seen = set()

        for row in rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                link = cols[1].find_element(By.TAG_NAME, "a").get_attribute("href")

                if link in seen:
                    continue
                seen.add(link)

                naac = cols[1].find_elements(By.CLASS_NAME, "naac-grade")
                fees = cols[2].find_elements(By.CLASS_NAME, "text-green")
                pkgs = cols[3].find_elements(By.CLASS_NAME, "text-green")
                ranking = cols[5].find_elements(By.CLASS_NAME, "rank-span")

                self.data.append({
                    "CD_Rank": cols[0].text.strip(),
                    "College_Name": cols[1].find_element(By.TAG_NAME, "h3").text.strip(),
                    "College_URL": link,
                    "Location": cols[1].find_element(By.CLASS_NAME, "location").text.strip(),
                    "Approvals": cols[1].find_element(By.CLASS_NAME, "approvals").text.strip(),
                    "NAAC_Grade": naac[0].text.strip() if naac else None,
                    "First_Year_Fees": fees[0].text.strip() if fees else None,
                    "Avg_Package": pkgs[0].text.strip() if pkgs else None,
                    "Highest_Package": pkgs[1].text.strip() if len(pkgs) > 1 else None,
                    "Rating": cols[4].find_element(By.CLASS_NAME, "lr-key").text.strip(),
                    "Reviews": cols[4].find_element(By.CLASS_NAME, "lr-value").text.strip(),
                    "National_Ranking": ranking[0].text.strip() if ranking else None
                })

            except Exception:
                continue

        driver.quit()
        return pd.DataFrame(self.data)


In [3]:
class CollegeDataCleaner:

    @staticmethod
    def _number(text):
        if pd.isna(text): 
            return None
        text = re.sub(r"[₹,$]", "", str(text))
        m = re.search(r"\d+(\.\d+)?", text)
        return float(m.group()) if m else None

    @staticmethod
    def _rating(text):
        if pd.isna(text): 
            return None
        m = re.search(r"\d+(\.\d+)?", str(text))
        return float(m.group()) if m else None

    @staticmethod
    def _rank(text):
        if pd.isna(text): 
            return None
        m = re.search(r"\d+", str(text).replace(",", ""))
        return int(m.group()) if m else None

    @classmethod
    def normalize(cls, df):
        df["First_Year_Fees"] = df["First_Year_Fees"].apply(cls._number)
        df["Avg_Package"] = df["Avg_Package"].apply(cls._number)
        df["Highest_Package"] = df["Highest_Package"].apply(cls._number)
        df["Rating"] = df["Rating"].apply(cls._rating)
        df["National_Ranking"] = df["National_Ranking"].apply(cls._rank)
        return df


In [4]:
class CollegeNameMatcher:
    def __init__(self, official_path, sheet_index):
        self.official = pd.read_excel(official_path, sheet_name=sheet_index)
        self.name_col = self._detect("name")
        self.code_col = self._detect("code")

        self.official["clean_name"] = self.official[self.name_col].apply(self._clean)
        self.choices = self.official["clean_name"].tolist()

    def _detect(self, key):
        for col in self.official.columns:
            if key in col.lower():
                return col
        raise ValueError(f"Column containing '{key}' not found")

    def _clean(self, text):
        text = "" if pd.isna(text) else text.lower()
        text = re.sub(r"\(.*?\)|\[.*?\]", "", text)
        text = re.sub(r"[^a-z0-9 ]", " ", text)
        return re.sub(r"\s+", " ", text).strip()

    def match(self, scraped_df):
        matched = []

        for _, row in scraped_df.iterrows():
            query = self._clean(row["College_Name"])

            result = process.extractOne(query, self.choices, scorer=fuzz.token_set_ratio)
            if not result:
                continue

            _, score, idx = result

            # ✅ KEEP ONLY PERFECT MATCHES
            if score == 100:
                official = self.official.iloc[idx]

                matched.append({
                    **row,
                    "college_code": official[self.code_col],
                    "Match_Score": 100
                })

        df = pd.DataFrame(matched)
        return CollegeDataCleaner.normalize(df)


In [5]:
scraper = CollegeScraper("https://collegedunia.com/btech/karnataka-colleges?exam_id=61")
raw_df = scraper.scrape()

matcher = CollegeNameMatcher(r"C:\Users\mahes\OneDrive\Desktop\kcet_companion\backend\kcet_backend\core\data\collegedunia_college_names.xlsx", sheet_index=3)
final_df = matcher.match(raw_df)

final_df.to_csv("karnataka_colleges_final.csv", index=False)
print("✅ Done. Saved to karnataka_colleges_final.csv")


✅ Done. Saved to karnataka_colleges_final.csv


In [6]:
final_df

,CD_Rank,College_Name,College_URL,Location,Approvals,NAAC_Grade,First_Year_Fees,Avg_Package,Highest_Package,Rating,Reviews,National_Ranking,college_code,Match_Score
0,#1,"R V College of Engineering - [RVCE], Bangalore",https://collegedunia.com/college/14673-r-v-col...,"Bangalore, Karnataka","AICTE, NBA Approved",A+,1332840.0,1252400.0,6700000.0,3.9,Based on 442 User\nReviews,85.0,E005,100
1,#2,"BMS College of Engineering - [BMSCE], Bangalore",https://collegedunia.com/college/5540-bms-coll...,"Bangalore, Karnataka","AICTE, NBA Approved",A++,1204400.0,1140000.0,5150000.0,4.1,Based on 366 User\nReviews,96.0,E003,100
2,#3,"PES University - [PESU], Bangalore",https://collegedunia.com/university/28294-pes-...,"Bangalore, Karnataka","AICTE, UGC Approved",A+,2210000.0,1010000.0,6500000.0,4.0,Based on 448 User\nReviews,140.0,E009,100
3,#4,"Ramaiah Institute of Technology - [RIT], Banga...",https://collegedunia.com/college/5621-ramaiah-...,"Bangalore, Karnataka","AICTE, NBA Approved",A+,1204400.0,766000.0,5000000.0,4.0,Based on 635 User\nReviews,143.0,E006,100
4,#5,Sri Jayachamarajendra College of Engineering -...,https://collegedunia.com/college/15219-sri-jay...,"Mysore, Karnataka","AICTE, NBA Approved",None,486440.0,5600000.0,NaN,4.1,Based on 96 User\nReviews,176.0,E021,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,#224,"Rai Technology University - [RTU], Bangalore",https://collegedunia.com/university/54849-rai-...,"Bangalore, Karnataka","UGC, AIU Approved",None,615000.0,NaN,NaN,4.0,Based on 22 User\nReviews,NaN,E256,100
200,#228,"CMR University (Lakeside Campus), Bangalore",https://collegedunia.com/university/65521-cmr-...,"Bangalore, Karnataka","AICTE, COA, UGC, AIU Approved",None,700000.0,550000.0,1800000.0,2.0,Based on 1 User\nReviews,NaN,E257,100
201,#230,"CMR University (City Campus - HRBR Layout), Ba...",https://collegedunia.com/university/56100-cmr-...,"Bangalore, Karnataka","AICTE, COA, BCI, UGC, AIU Approved",A,1450000.0,400000.0,1800000.0,3.8,Based on 659 User\nReviews,246.0,E257,100
202,#231,"GITAM, Bangalore",https://collegedunia.com/university/25600-gita...,"Bangalore, Karnataka","AICTE, UGC, MHRD Approved",A++,1384000.0,652000.0,1500000.0,3.9,Based on 136 User\nReviews,NaN,E255,100


In [ ]:
import pandas as pd

def clean_college_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans null values in the college dataframe with sensible defaults.
    Modifies and returns the dataframe.
    """

    fill_rules = {
        "College_Name": None,                 # keep as-is
        "Location": None,                     # keep as-is
        "Approvals": None,                    # keep as-is
        "NAAC_Grade": "No NAAC",
        "First_Year_Fees": 450000,
        "Avg_Package": 0,
        "Highest_Package": 0,
        "Rating": 3.5,
        "National_Ranking":0        # keep as-is
    }

    for column, default in fill_rules.items():
        if column in df.columns and default is not None:
            df[column] = df[column].fillna(default)

    return df


In [11]:
final_df = clean_college_dataframe(final_df)
final_df.to_excel("karnataka_colleges_finals.xlsx", index=False)

In [9]:
import os, sys
from pathlib import Path

# Reset any broken config
os.environ.pop("DJANGO_SETTINGS_MODULE", None)

# Add the folder that contains the INNER kcet_backend
PROJECT_ROOT = Path(r"C:\Users\mahes\OneDrive\Desktop\kcet_companion\backend\kcet_backend")
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added:", PROJECT_ROOT)


Project root added: C:\Users\mahes\OneDrive\Desktop\kcet_companion\backend\kcet_backend


In [10]:
os.environ["DJANGO_SETTINGS_MODULE"] = "kcet_backend.kcet_backend.settings"
print("Settings module:", os.environ["DJANGO_SETTINGS_MODULE"])


Settings module: kcet_backend.kcet_backend.settings


In [11]:
# 🔥 HARD RESET DJANGO + JUPYTER STATE

import sys, os
from pathlib import Path

# 1️⃣ Clear old Django state
for key in list(os.environ.keys()):
    if key.startswith("DJANGO_"):
        del os.environ[key]

# 2️⃣ Clear cached Django modules
for m in list(sys.modules.keys()):
    if m.startswith("django") or m.startswith("kcet_backend") or m.startswith("core"):
        del sys.modules[m]

# 3️⃣ Locate Django project automatically
p = Path.cwd()
while p != p.root and not (p / "manage.py").exists():
    p = p.parent

if not (p / "manage.py").exists():
    raise RuntimeError("❌ manage.py not found. Open notebook inside the project tree.")

sys.path.insert(0, str(p))

# 4️⃣ Detect project package
project_pkg = next(d.name for d in p.iterdir() if (d / "settings.py").exists())
os.environ["DJANGO_SETTINGS_MODULE"] = f"{project_pkg}.settings"

print("✅ Project root:", p)
print("✅ Settings:", os.environ["DJANGO_SETTINGS_MODULE"])


✅ Project root: c:\Users\mahes\OneDrive\Desktop\kcet_companion\backend\kcet_backend
✅ Settings: kcet_backend.settings


In [12]:
from core.services.ingestors.college_ingestor import CollegePreprocessor
from core.services.ingestors.college_ingestor import CollegeIngestor
print("Pipeline imports successful")


AppRegistryNotReady: Apps aren't loaded yet.

In [13]:


from core.services.ingestors.college_ingestor import CollegePreprocessor
from core.services.ingestors.college_ingestor import CollegeIngestor

processor = CollegePreprocessor(
    scrape_url="https://collegedunia.com/btech/karnataka-colleges?exam_id=61",
    official_excel=r"C:\Users\mahes\OneDrive\Desktop\kcet_companion\backend\kcet_backend\core\data\collegedunia_college_names.xlsx",
    sheet_index=3
)

df = processor.run()

ingestor = CollegeIngestor(df)
report = ingestor.ingest()


print(report)


AppRegistryNotReady: Apps aren't loaded yet.

In [27]:
final_df

,CD_Rank,College_Name,College_URL,Location,Approvals,NAAC_Grade,First_Year_Fees,Avg_Package,Highest_Package,Rating,Reviews,National_Ranking,college_code,Match_Score
0,#1,"R V College of Engineering - [RVCE], Bangalore",https://collegedunia.com/college/14673-r-v-col...,"Bangalore, Karnataka","AICTE, NBA Approved",A+,1332840.0,1252400.0,6700000.0,3.9,Based on 439 User\nReviews,85.0,E005,100
1,#2,"BMS College of Engineering - [BMSCE], Bangalore",https://collegedunia.com/college/5540-bms-coll...,"Bangalore, Karnataka","AICTE, NBA Approved",A++,1204400.0,1140000.0,5150000.0,4.1,Based on 366 User\nReviews,96.0,E003,100
2,#3,"PES University - [PESU], Bangalore",https://collegedunia.com/university/28294-pes-...,"Bangalore, Karnataka","AICTE, UGC Approved",A+,2210000.0,1010000.0,6500000.0,3.9,Based on 525 User\nReviews,140.0,E009,100
3,#4,"Ramaiah Institute of Technology - [RIT], Banga...",https://collegedunia.com/college/5621-ramaiah-...,"Bangalore, Karnataka","AICTE, NBA Approved",A+,1204400.0,766000.0,5000000.0,4.0,Based on 634 User\nReviews,143.0,E006,100
4,#5,Sri Jayachamarajendra College of Engineering -...,https://collegedunia.com/college/15219-sri-jay...,"Mysore, Karnataka","AICTE, NBA Approved",None,486440.0,5600000.0,NaN,4.1,Based on 96 User\nReviews,176.0,E021,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,#146,SEA College of Engineering and Technology - [S...,https://collegedunia.com/college/14821-sea-col...,"Bangalore, Karnataka","AICTE, NBA Approved",B++,880000.0,420000.0,NaN,3.5,Based on 40 User\nReviews,NaN,E088,100
139,#147,Mysore College of Engineering & Management - [...,https://collegedunia.com/college/57906-mysore-...,"Mysore, Karnataka",AICTE Approved,None,756768.0,NaN,NaN,3.1,Based on 7 User\nReviews,NaN,E252,100
140,#148,"Government Engineering College - [GEC], Karwar",https://collegedunia.com/college/59501-governm...,"Karwar, Karnataka",AICTE Approved,None,176800.0,NaN,NaN,4.2,Based on 7 User\nReviews,NaN,E002,100
141,#149,"Maratha Mandal Engineering College - [MMEC], B...",https://collegedunia.com/college/14280-maratha...,"Belgaum, Karnataka",AICTE Approved,None,449640.0,400000.0,1000000.0,3.0,Based on 9 User\nReviews,NaN,E029,100
